# Raw signature verification

## Parameters

### Field

In [2]:
p = 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319

### Curve

In [2]:
ca = p - 3
cb = 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575
gx = 26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087
gy = 8325710961489029985546751289520108179287853048861315594709205902480503199884419224438643760392947333078086511627871
go = 39402006196394479212279040100143613805079739270465446667946905279627659399113263569398956308152294913554433653942643

### Public key

In [3]:
x = 30827567487393551525402089664526449770646356836908584670673246523285555687049858098829191759343589369441587654976562
y = 2009068527738404759430491179267117270890161016114510417259692707866849715139082362044671225604392405754431155211363

### Digest

In [4]:
m = 29487358125424512498370551173628780853002944075174315746794686527735994862322

### Signature

In [5]:
r = 36742824419652648025863987403620927536315918738674799855291881974208903666047095304901170886918427806277713149969790
s = 36655663933123342092039028055267978704456567804916507890392767843061757921835920729532485652317488042964862515704415

## Curve math

In [6]:
def fieldAdd(a, b):
    return (a + b) % p

In [21]:
def fieldSub(a, b):
    return (a - b) % p

In [51]:
def fieldMul(a, b):
    print(a, b, (a * b) % p)
    print('------')
    return (a * b) % p

In [23]:
def fieldInv(a):
    a = a % p

    # Euclids
    c, d = a, p
    uc, vc, ud, vd = 1, 0, 0, 1
    while c != 0:
        q, c, d = (d // c, d % c, c)
        uc, vc, ud, vd = ud - q * uc, vd - q * vc, uc, vc
        
    assert d == 1
    if ud > 0:
        return ud
    else:
        return ud + p

In [24]:
def orderAdd(a, b):
    return (a + b) % go

In [25]:
def orderSub(a, v):
    return (a - b) % go

In [26]:
def orderMul(a, b):
    return (a * b) % go

In [27]:
def orderInv(a):
    a = a % go

    # Euclids
    c, d = a, go
    uc, vc, ud, vd = 1, 0, 0, 1
    while c != 0:
        q, c, d = (d // c, d % c, c)
        uc, vc, ud, vd = ud - q * uc, vd - q * vc, uc, vc
        
    assert d == 1
    if ud > 0:
        return ud
    else:
        return ud + p

In [52]:
def curveAdd(a, b):
    ax, ay = a
    bx, by = b
    # l = (ay - by) / (ax - bx)
    l = fieldMul(fieldSub(ay, by), fieldInv(fieldSub(ax, bx)))
    # x = l * l - ax - bx
    x = fieldSub(fieldMul(l, l), fieldAdd(ax, bx))
    # y = l * (bx - x) - by
    y = fieldSub(fieldMul(l, fieldSub(bx, x)), by)
    return (x, y)

In [53]:
def curveDouble(a):
    ax, ay = a
    # l = (3 * ax * ax + ca) / (2 * ay)
    l = fieldMul(fieldAdd(fieldMul(3, fieldMul(ax, ax)), ca), fieldInv(fieldMul(2, ay)))
    # x = l * l - 2 * ax
    x = fieldSub(fieldMul(l, l), fieldMul(2, ax))
    # y = l * (ax - x) - ay
    y = fieldSub(fieldMul(l, fieldSub(ax, x)), ay)
    return (x, y)

In [54]:
def curveMul(r, a):
    assert(r != 0)
    # Double and add
    
    result = False
    coeff = a
    while r > 0:
        if r % 2 == 1:
            if result == False:
                result = coeff
            else:
                result = curveAdd(result, coeff)
        coeff = curveDouble(coeff)
        r = r // 2
    return result

## ECDSA

In [55]:
def sign()

SyntaxError: invalid syntax (<ipython-input-55-d0819e8401f6>, line 1)

In [56]:
def verify(pub, m, r, s):
    c = orderInv(s)
    u1 = orderMul(m, c)
    u2 = orderMul(r, c)
    t1 = curveMul(u1, (gx, gy))
    t2 = curveMul(u2, pub)
    x, y = curveAdd(t1, t2)
    return x == r

In [57]:
verify((x, y), m, r, s)

26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087 26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087 679971533856132212205617414932360354764106180264079082911937676692986189053294113671889426697513798718359702384665
------
3 679971533856132212205617414932360354764106180264079082911937676692986189053294113671889426697513798718359702384665 2039914601568396636616852244797081064292318540792237248735813030078958567159882341015668280092541396155079107153995
------
2 8325710961489029985546751289520108179287853048861315594709205902480503199884419224438643760392947333078086511627871 16651421922978059971093502579040216358575706097722631189418411804961006399768838448877287520785894666156173023255742
------
2039914601568396636616852244797081064292318540792237248735813030078958567159882341015668280092541396155079107153992 3193191483210484360271527117108549473427899961

------
3 25176575955795651931541733095696713185976746132982805868928772919298948303169049480305600952644042708925986545892237 36127721670992476582346159186946525752850499128482970938838025353651123138010278111869536769673190124916352664564392
------
2 29761170505703847994989742723227292736743024483730818175817863758191127478833361126483844803542142315198502007378406 20120334815013216777700445346310971668406309696996189683687434112136533186169851923920423518825346628535397041644493
------
36127721670992476582346159186946525752850499128482970938838025353651123138010278111869536769673190124916352664564389 16893243495387033938004151194089202885223129191187455349781042870763258121623856470369798222147959017710163439518975 12894715330016938654206453476228147942293794968447070142404300506428078852957988348724296953440709777702728469564889
------
12894715330016938654206453476228147942293794968447070142404300506428078852957988348724296953440709777702728469564889 12894715330016938654206453476228

------
3 30747828616494433040622678465746969519918125004383914625337334153355250692839823717376890390274054365910052527916170 13439473456694340697309955196953680949594896472220850540115415651574308535525730494036138994304287094006943637523872
------
2 16478413345363691670778457252598848007608455134058961193427646482596812019366238711153058376622806407596623808956718 32956826690727383341556914505197696015216910268117922386855292965193624038732477422306116753245612815193247617913436
------
13439473456694340697309955196953680949594896472220850540115415651574308535525730494036138994304287094006943637523869 17953746589538922014448487742117217956092913331451716697353269024836752212078820298284173303917538906311995562794845 18648691467602206896560844611496233185451352567848100524855980662752249242350600003684035012064714870383846139679318
------
18648691467602206896560844611496233185451352567848100524855980662752249242350600003684035012064714870383846139679318 18648691467602206896560844611496

------
1802759182260869429946947126933885525566624486789384767910905268754741419209374229103874190834448889418967332968251 22588964232610998879864874901796713879192661067064576899006446639470718940031474069216911148887981814168500166655758 26849854649479069191448015890468313247815844461695683071364072702646141148251782402794666658124312321787931130692254
------
26849854649479069191448015890468313247815844461695683071364072702646141148251782402794666658124312321787931130692254 26849854649479069191448015890468313247815844461695683071364072702646141148251782402794666658124312321787931130692254 11100896852416415477200551582578975741353926128324310009960630453740785488722721220698055812795934421667847248987726
------
2 28299987771706758673168092898495916388627158717527894876593530994336576405427525008822222931158155901586731138874193 17197969347019038134057145696848218972174578164590343085238768584427431039358179688597179774057373801311855304636067
------
26849854649479069191448015890468313

22452960304179607961635549091979503744805586590305576748308273963880859918636550503732622124903275215736164919755702 33301003798634542518848170725718976965820571274473243694306453501184048440697031969155370144861347167560067171165447 24362901304231679866826732265629206372138172745329340928706246572144277484487188803145344147826096516537442033439198
------
24362901304231679866826732265629206372138172745329340928706246572144277484487188803145344147826096516537442033439198 24362901304231679866826732265629206372138172745329340928706246572144277484487188803145344147826096516537442033439198 28977152844191868079276830638595209363718382922262007448469869393887532219024944642231585695082072591993410316271241
------
24362901304231679866826732265629206372138172745329340928706246572144277484487188803145344147826096516537442033439198 24262458579739784669413147509175887299129901671843862630254710075454717218324760543936317715932792177666798286875097 63318910337755312063597017805978582516786893593840

...

1658964947515726828078176478263926626349728845585833012295451403185393982168078964144826988381699055601664298508055 1658964947515726828078176478263926626349728845585833012295451403185393982168078964144826988381699055601664298508055 14569066433099175520752083399809655153859959782844041350915234019451006743800007773585577679117173494137919638738488
------
3 14569066433099175520752083399809655153859959782844041350915234019451006743800007773585577679117173494137919638738488 4305193102903047349977210099285351656500140078066677384797408654107298459903152991709466949092582480552151943103145
------
2 12743979473716220413391303383057178072658689061440787435444417767522981226066561265968039453939061892602948230586270 25487958947432440826782606766114356145317378122881574870888835535045962452133122531936078907878123785205896461172540
------
4305193102903047349977210099285351656500140078066677384797408654107298459903152991709466949092582480552151943103142 3313195261536013628204548995743910927216880

------
21169558254890682700276737207959047808585854845136431814380619694987299805291644703725802831366777390675509042290896 30917187277802076766991433990542466936433908034915869186874934173949904310526417807615799884616295922230831817136958 3286492476463538766831179804475470779734878995078502207125106015056523546980331104703928097754712516470019492429829
------
1768513260011862671111806813043309052225948449602067138756746125931510426111245296391177539748557977329687946495963 1768513260011862671111806813043309052225948449602067138756746125931510426111245296391177539748557977329687946495963 29563809899147793501163790323494587492054491922034494093499645525531403693227100550987219120603791042949612972525325
------
3 29563809899147793501163790323494587492054491922034494093499645525531403693227100550987219120603791042949612972525325 9887417304654422078933290770196534866003997225172588944602349768102767536687560994867125185293497125125624971351337
------
2 196020114741228869032261914114569254

2 18836587146211553804147831820311995604218895740279607626393918217828697812033416963402559353304268066788613607293676 37673174292423107608295663640623991208437791480559215252787836435657395624066833926805118706608536133577227214587352
------
10160642743948778508737254858253281518592073387608680155514418061986389592631301197353783346664271171809912013548751 38090470326362266080891973237060408611043070635061751159569007025268493769023789643794163097897116242990865443521448 3979389466892859266082039491190866757364469291032629009159384121876465224310969754703163940330553662259829230040720
------
3979389466892859266082039491190866757364469291032629009159384121876465224310969754703163940330553662259829230040720 3979389466892859266082039491190866757364469291032629009159384121876465224310969754703163940330553662259829230040720 37649175918238551161413369295383288100571359733288175434839968390628733479469829015716293212584308904933220205541023
------
2 139761384970202604027994507671314736578042

------
20944420901028178744440360722286201262315749822576435486775154988543232259335052858200620733210138489702053724461006 15100175535898882775525603607588230718171825591337047318111293143808062289842574354192386580072645819991712715280573 15087918615856179916602697437373114451997121314024255515250716134060722521243140283583438040251397983348171879976123
------
34658726188351394970971496996470452340305144179507150408187713592778691842064425098187157170186103699840083680845441 34658726188351394970971496996470452340305144179507150408187713592778691842064425098187157170186103699840083680845441 33721683491531172477017329370224084316849025414578179667793039893071345803061243001539884325782705536430377888377323
------
3 33721683491531172477017329370224084316849025414578179667793039893071345803061243001539884325782705536430377888377323 22361038081804559006493907910385025340387597702803645667482532870722593866189988346525120800830240605567919718907331
------
2 15356395544109646642383236784979

True

## 256 bit math

In [36]:
B = 2**256

In [37]:
phi = p // B
plo = p % B

In [40]:
hex(phi)

'0xffffffffffffffffffffffffffffffff'

In [41]:
hex(plo)

'0xfffffffffffffffffffffffffffffffeffffffff0000000000000000ffffffff'

In [46]:
def encode(n):
    hi = n // B
    lo = n % B
    return(hex(hi), hex(lo))

In [47]:
encode(p - 2)

('0xffffffffffffffffffffffffffffffff',
 '0xfffffffffffffffffffffffffffffffeffffffff0000000000000000fffffffd')

In [45]:
384*4*15000

23040000

In [3]:
hex(p)

'0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffeffffffff0000000000000000ffffffff'

In [4]:
p - 2**384

-340282367000166625977638945021017194497

In [5]:
hex(2**256 -1)

'0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'